**Dynamic Orbit CSV Generator**

This notebook creates CSV files of stellar orbital positions in either a galactocentric or a heliocentric refrence frame.

In [ ]:
OBJECT_NAME = 'Orion_YSOs_Heliocentric'
FILE_LOCATION = 'drive/Shared drives/Owen/Orion_XYZ_UVW_200-800pc_YSOs_more-columns.fits'

REGRESSIVE   = True
HELIOCENTRIC = True
SAMPLE_SIZE  = 10000
TIMESPAN     = 20000000
TIMESTEP     = 100000

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip -q install astropy
from astropy.io import fits
from astropy.table import Table
from astropy import units as u
from astropy import coordinates as coord
from astropy.coordinates import SkyCoord
from astropy.coordinates import LSR

!pip -q install galpy
import galpy
from galpy.orbit import Orbit
from galpy.potential import MWPotential2014

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
%matplotlib inline

from google.colab import files

In [ ]:
file = fits.open(FILE_LOCATION, memmap=False)

print(file.info(),'\n-----------\n',file[1].columns)
#file.close()

Filename: drive/Shared drives/Owen/Orion_XYZ_UVW_200-800pc_YSOs_more-columns.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU      16   (1652,)   uint8   
  1  /Users/josefa/Dropbox/PAPERS/04_CO_UVW/fits/Orion_XYZ_UVW_200-800...    1 BinTableHDU     77   1898R x 25C   [20A, D, D, D, D, 4A, D, D, D, D, D, D, 20A, D, D, D, D, D, D, D, D, D, D, D, D]   
None 
-----------
 ColDefs(
    name = 'gaia_source_id'; format = '20A'
    name = 'ra'; format = 'D'; unit = 'deg'
    name = 'dec'; format = 'D'; unit = 'deg'
    name = 'l_gaia'; format = 'D'; unit = 'deg'
    name = 'b_gaia'; format = 'D'; unit = 'deg'
    name = 'Class'; format = '4A'
    name = 'plx'; format = 'D'; unit = 'mas'
    name = 'plx_err'; format = 'D'; unit = 'mas'
    name = 'pmra'; format = 'D'
    name = 'pmdec'; format = 'D'
    name = 'pmra_err'; format = 'D'
    name = 'pmdec_err'; format = 'D'
    name = 'rv_ref'; format = '20A'
    name = 'rv'; format = 'D'
    name

In [ ]:
df = pd.DataFrame({'ra':[], 'dec':[], 'distance':[], 'U':[], 'V':[], 'W':[]})
for star in file[1].data:
  if (np.isnan(star.field('U')) or np.isnan(star.field('V')) or np.isnan(star.field('W'))) == False:
    df = df.append({'ra':star.field('RA'),
                    'dec':star.field('DEC'),
                    'distance': (1/(star.field('plx')/1000)),
                    'U':star.field('U'),
                    'V':star.field('V'),
                    'W':star.field('W')},
                    ignore_index=True)

In [ ]:
orbits = []
orbits.append(Orbit())

for index, row in df.iterrows():
  o = Orbit(vxvv=[row['ra'] * u.deg,
                  row['dec'] * u.deg,
                  row['distance']/1000 * u.kpc,
                  row['U'] * u.km/u.s,
                  row['V'] * u.km/u.s,
                  row['W'] * u.km/u.s],
                  radec=True,
                  uvw=True,
                  solarmotion=u.Quantity([-LSR.v_bary.d_x, LSR.v_bary.d_y, LSR.v_bary.d_z]))
  orbits.append(o)

In [ ]:
ts = np.linspace(0,TIMESPAN,int(TIMESPAN/TIMESTEP)) * u.yr
for orbit in orbits:
  if (REGRESSIVE == True):
    orbit.flip(inplace=True)
  orbit.integrate(ts, MWPotential2014, method = 'dopr54_c')

In [ ]:
file_index = int(TIMESPAN/TIMESTEP)-1 

for t in range(0,TIMESPAN,TIMESTEP):
  df_temp = pd.DataFrame({'x coord':[], 'y coord':[], 'z coord':[]})
  for orbit in orbits:
    if (HELIOCENTRIC == True):
      df_temp = df_temp.append({'x coord':-1000*((orbit.x(t*u.yr)-orbits[0].x(t*u.yr))),
                                'y coord':1000*(orbit.y(t*u.yr)-orbits[0].y(t*u.yr)),
                                'z coord':1000*(orbit.z(t*u.yr)-orbits[0].z(t*u.yr))},
                                ignore_index=True)
    else:
      df_temp = df_temp.append({'x coord':-1000*orbit.x(t*u.yr),
                                'y coord':1000*orbit.y(t*u.yr),
                                'z coord':1000*orbit.z(t*u.yr)},
                                ignore_index=True)

  file_name = OBJECT_NAME + "." + str(file_index) + ".csv"
  df_temp.to_csv(path_or_buf=file_name, index=False)
  files.download(file_name)
  
  if (REGRESSIVE == True): file_index -= 1
  else: file_index += 1

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>